In [ ]:
!pip install kaggle tensorflow numpy opencv-python matplotlib

# Import necessary libraries
import os
import zipfile
import shutil
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from google.colab import drive

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
import shutil

# Create directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json
shutil.move("kaggle.json", "/root/.kaggle/")

# Change permission
os.chmod("/root/.kaggle/kaggle.json", 600)


In [ ]:
!kaggle datasets list -s "real-and-fake-face-detection"


In [ ]:
# Authenticate Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d ciplab/real-and-fake-face-detection -p /content/


In [ ]:
import zipfile
import os

# Define paths
DOWNLOAD_PATH = "/content/"
EXTRACTION_PATH = "/content/real_fake_faces"

# Extract dataset
zip_files = [f for f in os.listdir(DOWNLOAD_PATH) if f.endswith('.zip')]
for file in zip_files:
    with zipfile.ZipFile(os.path.join(DOWNLOAD_PATH, file), 'r') as zip_ref:
        zip_ref.extractall(EXTRACTION_PATH)

print("✅ Dataset extracted successfully!")


In [ ]:
import os
import shutil

# Paths
EXTRACTED_PATH = "/content/real_fake_faces"
DATASET_FOLDER_1 = os.path.join(EXTRACTED_PATH, "real_and_fake_face")
DATASET_FOLDER_2 = os.path.join(EXTRACTED_PATH, "real_and_fake_face_detection")

# Define new structured dataset path
MAIN_DATASET_PATH = "/content/structured_deepfake_dataset"
REAL_PATH = os.path.join(MAIN_DATASET_PATH, "real")
FAKE_PATH = os.path.join(MAIN_DATASET_PATH, "fake")

# Ensure folders exist
os.makedirs(REAL_PATH, exist_ok=True)
os.makedirs(FAKE_PATH, exist_ok=True)


In [ ]:
# Move images based on dataset structure
for dataset_folder in [DATASET_FOLDER_1, DATASET_FOLDER_2]:
    for subfolder in os.listdir(dataset_folder):
        subfolder_path = os.path.join(dataset_folder, subfolder)

        if "real" in subfolder.lower():
            for file in os.listdir(subfolder_path):
                shutil.move(os.path.join(subfolder_path, file), REAL_PATH)

        elif "fake" in subfolder.lower():
            for file in os.listdir(subfolder_path):
                shutil.move(os.path.join(subfolder_path, file), FAKE_PATH)

print(" Dataset is now structured into 'real/' and 'fake/' folders!")


In [ ]:
print("Real images count:", len(os.listdir(REAL_PATH)))
print("Fake images count:", len(os.listdir(FAKE_PATH)))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    MAIN_DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training'
)

val_data = datagen.flow_from_directory(
    MAIN_DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation'
)

print("Dataset loaded successfully for training!")


In [ ]:
# Create Kaggle API directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d ciplab/real-and-fake-face-detection -p /content/

# Extract dataset
import zipfile
import os

EXTRACTION_PATH = "/content/real_fake_faces"

zip_files = [f for f in os.listdir("/content/") if f.endswith('.zip')]
for file in zip_files:
    with zipfile.ZipFile(os.path.join("/content/", file), 'r') as zip_ref:
        zip_ref.extractall(EXTRACTION_PATH)

print("Dataset downloaded and extracted!")


In [ ]:
import os
import shutil

# Paths
DATASET_FOLDER_1 = os.path.join(EXTRACTION_PATH, "real_and_fake_face")
DATASET_FOLDER_2 = os.path.join(EXTRACTION_PATH, "real_and_fake_face_detection")

# New structured dataset path
MAIN_DATASET_PATH = "/content/structured_deepfake_dataset"
REAL_PATH = os.path.join(MAIN_DATASET_PATH, "real")
FAKE_PATH = os.path.join(MAIN_DATASET_PATH, "fake")

# Ensure folders exist
os.makedirs(REAL_PATH, exist_ok=True)
os.makedirs(FAKE_PATH, exist_ok=True)

# Move images safely
def move_files(dataset_folder, category, target_path):
    subfolder_path = os.path.join(dataset_folder, category)

    if not os.path.exists(subfolder_path):
        return

    for file in os.listdir(subfolder_path):
        src_file = os.path.join(subfolder_path, file)
        dst_file = os.path.join(target_path, file)

        # Avoid duplicate errors
        if os.path.exists(dst_file):
            base, ext = os.path.splitext(file)
            new_filename = f"{base}_copy{ext}"  # Rename duplicate files
            dst_file = os.path.join(target_path, new_filename)

        shutil.move(src_file, dst_file)

# Move files from both dataset folders
for dataset_folder in [DATASET_FOLDER_1, DATASET_FOLDER_2]:
    for category in os.listdir(dataset_folder):
        if "real" in category.lower():
            move_files(dataset_folder, category, REAL_PATH)
        elif "fake" in category.lower():
            move_files(dataset_folder, category, FAKE_PATH)

print("✅ Dataset structured successfully without duplicate errors!")



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load Training Data
train_data = datagen.flow_from_directory(
    MAIN_DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training'
)

# Load Validation Data
val_data = datagen.flow_from_directory(
    MAIN_DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation'
)

print("✅ Data loaded successfully!")



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0

# Load EfficientNetB0 as base model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Build classifier
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')  # Binary classification (Real vs Fake)
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("✅ Model built successfully!")



In [ ]:
EPOCHS = 10

history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# Save model for future use
model.save("/content/deepfake_detector.h5")

print("✅ Model trained and saved successfully!")

In [ ]:
import matplotlib.pyplot as plt

# Accuracy plot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Training Accuracy")
plt.show()

# Loss plot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Training Loss")
plt.show()

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("/content/model.h5")  # Ensure the correct model path

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (128, 128))  # Resize to model input size
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)[0][0]
    label = "Fake" if prediction > 0.5 else "Real"
    print(f"Prediction: {label} (Confidence: {prediction:.2f})")

# Test with an image
predict_image("/content/real_00140.jpg")  # Replace with your image path







In [ ]:
import cv2
import numpy as np

# Function to predict if an image is real or fake
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE[::-1])  # OpenCV uses (width, height)
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)[0][0]
    label = "Fake" if prediction  > 0.76 else "Real"
    print(f"Prediction: {label} (Confidence: {prediction:.2f})")

# Test with an image
predict_image("/content/real_00140.jpg")  # Replace with your image path


